In [25]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image

In [26]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = read_image(img_path).float() / 255.0  # Normalize to [0,1]
        if self.transform:
            image = self.transform(image)
        return image, img_path


In [27]:
source_directory = "D:/MED_LEAF_ID/dataset/Medicinal Leaf dataset"
augmented_directory = "D:/MED_LEAF_ID/data/augmented"
os.makedirs(augmented_directory, exist_ok=True)


In [28]:
transform = transforms.Compose([
    transforms.RandomRotation(40),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.GaussianBlur(kernel_size=(3, 3)),])

In [29]:
TARGET_AUGMENTED_COUNT = 500 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [30]:
import torch

print("Torch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0))
print("Current Device:", torch.cuda.current_device())
print("CUDA Memory Allocated:", torch.cuda.memory_allocated(0) / 1024**2, "MB")
print("CUDA Memory Reserved:", torch.cuda.memory_reserved(0) / 1024**2, "MB")


Torch Version: 2.6.0+cu126
CUDA Available: True
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU
Current Device: 0
CUDA Memory Allocated: 773.275390625 MB
CUDA Memory Reserved: 1194.0 MB


In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a simple model
model = torch.nn.Linear(10, 2).to(device)

# Create a tensor and move it to GPU
x = torch.randn(10).to(device)
output = model(x)

print("Output Device:", output.device)  # Should print "cuda:0"


Output Device: cuda:0


In [32]:
import torch
import time

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create a large tensor and move it to GPU
tensor_size = (10000, 10000)
t = torch.randn(tensor_size, device=device)

# Measure computation time
start = time.time()
t.matmul(t)
torch.cuda.synchronize()  # Ensure all GPU operations finish
end = time.time()

print(f"Matrix multiplication time on GPU: {end - start:.6f} seconds")


Using device: cuda
Matrix multiplication time on GPU: 0.470891 seconds


In [33]:
import torch

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0))
print("Current Device:", torch.cuda.current_device())

tensor = torch.randn(10000, 10000, device="cuda")  # Allocate a large tensor on GPU
print("Tensor allocated successfully on:", tensor.device)


PyTorch Version: 2.6.0+cu126
CUDA Available: True
Device Name: NVIDIA GeForce RTX 3050 Laptop GPU
Current Device: 0
Tensor allocated successfully on: cuda:0


In [34]:
torch.backends.cudnn.benchmark = True


In [36]:
for plant_name in os.listdir(source_directory):
    class_folder = os.path.join(source_directory, plant_name)
    augmented_class_folder = os.path.join(augmented_directory, plant_name)
    os.makedirs(augmented_class_folder, exist_ok=True)

    existing_augmented_count = len([f for f in os.listdir(augmented_class_folder) if f.startswith("aug_")])
    if existing_augmented_count >= TARGET_AUGMENTED_COUNT:
        print(f"Skipping {plant_name}, already has {existing_augmented_count} augmented images.")
        continue

    print(f"Processing {plant_name}: {existing_augmented_count}/{TARGET_AUGMENTED_COUNT} augmented images present.")
    images_needed = TARGET_AUGMENTED_COUNT - existing_augmented_count

    image_paths = [os.path.join(class_folder, img) for img in os.listdir(class_folder) if img.endswith(('.jpg', '.png', '.jpeg'))]
    dataset = CustomImageDataset(image_paths, transform=transform)  # Apply transform here
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

    i = existing_augmented_count 
    for img_tensor, img_path in dataloader:
        img_tensor = img_tensor.to(device)  
        augmented_img = transform(img_tensor)  

        save_image(augmented_img.cpu(), os.path.join(augmented_class_folder, f"aug_{i}.png")) 
        i += 1
        if i >= TARGET_AUGMENTED_COUNT:
            break

print("Augmentation completed with PyTorch and GPU acceleration.")

Processing Aloevera: 0/500 augmented images present.
Processing Amla: 0/500 augmented images present.
Processing Amruthaballi: 0/500 augmented images present.
Processing Arali: 0/500 augmented images present.
Processing ashoka: 0/500 augmented images present.
Processing Astma_weed: 0/500 augmented images present.
Processing Badipala: 0/500 augmented images present.
Processing Balloon_Vine: 0/500 augmented images present.
Processing Bamboo: 0/500 augmented images present.
Processing Beans: 0/500 augmented images present.
Processing Betel: 0/500 augmented images present.
Processing Bhrami: 0/500 augmented images present.
Processing Bringaraja: 0/500 augmented images present.
Processing camphor: 0/500 augmented images present.
Processing Caricature: 0/500 augmented images present.
Processing Castor: 0/500 augmented images present.
Processing Catharanthus: 0/500 augmented images present.
Processing Chakte: 0/500 augmented images present.
Processing Chilly: 0/500 augmented images present.
P